<a href="https://colab.research.google.com/github/descandon88/tesis_maestria/blob/main/analisis_mozambique/feasible_products_proposition_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
# Librerías para visualizacion
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import re
import networkx as nx
from itertools import count
from itertools import combinations
from itertools import product
import statsmodels.api as sm
import urllib.request, json
!pip3 install geopandas
import geopandas
# -- set scientific notation to display numbers fully rather than exponential
pd.set_option('display.float_format', '{:.2f}'.format)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' 
import seaborn as sns
sns.set_style('whitegrid') # grids en dark background

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth',300)
print('librerías cargadas')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.0 MB/s eta 0:00:00
librerías cargadas


### Importacion de la bases desde drive



Se vincula la carpata de las bases con drive

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
## Se importa la base de exportaciones de productos drive
df_base = pd.read_csv("/content/drive/MyDrive/eci/base/df_updated.csv")
df_short_name = pd.read_csv("/content/drive/MyDrive/eci/base/df_tipo_pro.csv")

print('dataframes cargados')

dataframes cargados


In [5]:
df_base.describe()

,year,product_code,export_value
count,7373271.00,7373271.00,7373271.00
mean,2007.65,5304.58,42867657.55
std,7.44,2767.02,899054286.46
min,1995.00,101.00,0.00
25%,2001.00,2843.00,0.00
50%,2008.00,5402.00,1119.00
75%,2014.00,8102.00,432040.50
max,2020.00,9999.00,371679199232.00


In [6]:
# df_base['product_code'] = df_base['product_code'].apply(lambda x: '{0:0>4}'.format(x))
# df['product_code']  = df['product_code'] .apply(np.int64)

df_base.sample(n=6)
df_base.info()

,year,country_code,country_name,product_code,product_name,export_value
1140138,1996,EGY,Egypt,2304,"Oil-cake and other solid residues; whether or not ground or in the form of pellets, resulting from the extraction of soya-bean oil",3533
7024778,2007,MSR,Montserrat,9110,"Watch or clock movements, complete, unassembled or partly assembled (movement sets); incomplete watch or clock movements, assembled; rough watch or clock movements",0
558944,2007,CZE,Czech Republic,1101,Wheat or meslin flour,3208666
3189976,2011,AUS,Australia,4703,"Chemical wood pulp, soda or sulphate, other than dissolving grades",6477630
2289860,2020,VCT,Saint Vincent and the Grenadines,3214,"Glaziers' putty, grafting putty, resin cements, caulking compounds and other mastics; painters' fillings; non-refractory surfacing preparations for facades, indoor walls, floors, ceilings or the like",0
150761,2000,FJI,Fiji,403,"Buttermilk, curdled milk and cream, yoghurt, kephir, fermented or acidified milk or cream, whether or not concentrated, containing added sugar, sweetening matter, flavoured or added fruit or cocoa",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7373271 entries, 0 to 7373270
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   year          int64 
 1   country_code  object
 2   country_name  object
 3   product_code  int64 
 4   product_name  object
 5   export_value  int64 
dtypes: int64(3), object(3)
memory usage: 337.5+ MB


####Finding specific countries / products based on partial string matching If you’re interested in finding data on certain countries / products but not sure how exactly these are spelled in the data (or are spelled with / without e.g. capital letters)

In [7]:
STRING = 'Netherland'
df_base[df_base['country_name'].str.contains(STRING)][['country_name']].drop_duplicates()

STRING = 'wine'
df_base[df_base['product_name'].str.contains(STRING,flags=re.IGNORECASE, regex=True)][['product_name']].drop_duplicates()

,country_name
156,Netherlands Antilles
3711,Netherlands


,product_name
11924,Swine; live
47696,"Meat of swine; fresh, chilled or frozen"
65582,"Edible offal of bovine animals, swine, sheep, goats, horses, asses, mules or hinnies; fresh, chilled or frozen"
1084954,"Wine of fresh grapes, including fortified wines; grape must other than that of heading no. 2009"
1090916,"Vermouth and other wine of fresh grapes, flavoured with plants or aromatic substances"
1156498,Wine lees; argol
3855058,"Twine, cordage, ropes and cables, whether or not plaited or braided; whether or not impregnated, coated, covered or sheathed with rubber or plastics"
3861020,"Twine, cordage or rope; knotted netting, made up fishing nets and other made up nets, of textile materials"
3866982,"Articles of yarn, strip or the like of heading no. 5404 or 5405; twine, cordage, rope or cables n.e.s. or included"
4302208,"Rags; used or new, scrap twine, cordage, rope and cables and worn out articles of twine, cordage, rope or cables, of textile materials"


### Cálculo de Complejidad Económica

In [10]:
!pip install ecomplexity
print('py-ecomplexity instalado')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ecomplexity: filename=ecomplexity-0.5.2-py3-none-any.whl size=11283 sha256=e7018862280a42c5aa488966d8b071d802e9c4b1a7299428b2cc9959134af38e
  Stored in directory: /root/.cache/pip/wheels/a9/f4/30/1518af0a20cc7b77a37d94f9725d32912ef9a628e8d3a59bcc
Successfully built ecomplexity
py-ecomplexity instalado


In [11]:
from ecomplexity import ecomplexity
from ecomplexity import proximity

# To use py-ecomplexity, specify the following columns
# cols_input: dict of column names for time, location, product and value.
    #Example: {'time':'year', 'loc':'origin', 'prod':'hs92', 'val':'export_val'}
    
trade_cols = {'time':'year',
              'loc':'country_name',
              'prod':'product_name',
              'val':'export_value'}

print('Calculando complejidad económica')
df_ec = ecomplexity(df_base, trade_cols)
print('Cálculo finalizado')



Calculando complejidad económica
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Cálculo finalizado


In [12]:
df_complexity = df_ec[['country_name',
               'product_name',
               'product_code',
               'export_value',
               'rca',
               'mcp',
               'density',
               'year',
               'pci',
               'eci',
               'coi',
               'cog']].copy()

In [13]:
# Muestro resultados
df_complexity.head(n=8)

,country_name,product_name,product_code,export_value,rca,mcp,density,year,pci,eci,coi,cog
0,Afghanistan,"Abrasive powder or grain; natural or artificial, on a base of textile material, of paper, paperboard or of other material, whether or not cut to shape or sewn or otherwise made up",6805,1327,0.05,0,0.04,1995,3.01,-0.33,-0.59,1.37
1,Afghanistan,Accordions and similar instruments; mouth organs,9204,0,0.00,0,0.03,1995,3.88,-0.33,-0.59,1.05
2,Afghanistan,"Acetals and hemiacetals; whether or not with other oxygen function, and their halogenated, sulphonated, nitrated or nitrostated derivatives",2911,0,0.00,0,0.03,1995,3.77,-0.33,-0.59,1.42
3,Afghanistan,"Acids; carboxylic acid with additional oxygen function and their anhydrides, halides, peroxides, peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2918,0,0.00,0,0.04,1995,2.37,-0.33,-0.59,0.99
4,Afghanistan,"Acids; polycarboxylic acids, their anhydrides, halides, peroxides and peroxy-acids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2917,0,0.00,0,0.05,1995,1.50,-0.33,-0.59,0.74
5,Afghanistan,"Acids; saturated acyclic monocarboxylic acids and their anhydrides, halides, peroxides and peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2915,0,0.00,0,0.03,1995,3.03,-0.33,-0.59,1.30
6,Afghanistan,"Acids; unsaturated acyclic monocarboxylic, cyclic monocarboxylic , their anhydrides, halides, peroxides and peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2916,0,0.00,0,0.03,1995,3.33,-0.33,-0.59,1.51
7,Afghanistan,Acrylic polymers in primary forms,3906,0,0.00,0,0.03,1995,3.76,-0.33,-0.59,1.19


### Se realizan las siguientes tablas:


1.   Tabla con la distancia de productos para el 2014
2.   Tabla de rca de Mozambique para el 2014
3.   Tabla de OGI en el 2014




#### Cálculos de distancias(2014) y cálculos de rca(2014)

In [14]:
prepoThree2014 = df_complexity[df_complexity['year']==2014].copy()
# rcaMoz_2014 =  df_complexity[(df_complexity['year']==2014)&(df_complexity['country_name']=='Mozambique')].copy()

prepoThree2014 = prepoThree2014[['country_name','product_name','product_code','density','rca','cog','pci','year']].copy()

def Dexport_rca_mozam(data):
    df_all = data.copy()
    # df_all['rca_Mozambique_2014'] = df_all.apply(lambda x: (x.rca),axis =1 )
    df_all['rca_2014'] = df_all.apply(lambda x: (x.rca),axis =1 )

    df_all['Dexport_dummy'] = df_all.apply(lambda x: 1 if (x.rca_2014>0.1 and x.rca_2014 <0.99) else 0 ,axis=1)

    df_all.drop(['product_name','density','eci','coi'],axis=1,inplace=True,errors='ignore')
    df_all = df_all.reset_index(drop=True)

    print("tabla filtrada!")
    return df_all


def distance_calc(data):
  df_dist = data.copy()
  df_dist['distancia_2014'] = df_dist.apply(lambda x: (1-x.density),axis =1 )
  df_dist['rca_2014'] = df_dist.apply(lambda x: x.rca,axis =1 )
  df_dist['pci_2014'] = df_dist.apply(lambda x: x.pci,axis =1 )


  df_dist.drop(['product_name','year'],axis=1,inplace=True,errors='ignore')
  df_dist = df_dist.reset_index(drop=True)
  print("Calculo de distancia terminado!")
  return df_dist

# t_rcaMozam_2014 = Dexport_rca_mozam(rcaMoz_2014)
t_rca_2014 = Dexport_rca_mozam(prepoThree2014)
t_Distance2014 = distance_calc(prepoThree2014)

# t_rcaMozam_2014.head()
t_rca_2014.head()
t_Distance2014.head()

# t_Distance2014['product_code'].nunique()
# rcaMozam_2014['country_name'].nunique()

count_ones = (t_rca_2014['Dexport_dummy'] == 1).sum()
count_zeros = (t_rca_2014['Dexport_dummy'] == 0).sum()
# count_zeros_export_value = (t_rca_2014['export_value'] == 0).sum()

tabla filtrada!
Calculo de distancia terminado!


,country_name,product_code,rca,cog,pci,year,rca_2014,Dexport_dummy
0,Afghanistan,6805,0.00,1.40,3.72,2014,0.00,0
1,Afghanistan,2911,0.00,1.01,1.89,2014,0.00,0
2,Afghanistan,2918,0.00,0.79,1.99,2014,0.00,0
3,Afghanistan,2917,0.00,0.73,2.34,2014,0.00,0
4,Afghanistan,2915,0.00,0.98,3.05,2014,0.00,0


,country_name,product_code,density,rca,cog,pci,distancia_2014,rca_2014,pci_2014
0,Afghanistan,6805,0.04,0.00,1.40,3.72,0.96,0.00,3.72
1,Afghanistan,2911,0.03,0.00,1.01,1.89,0.97,0.00,1.89
2,Afghanistan,2918,0.05,0.00,0.79,1.99,0.95,0.00,1.99
3,Afghanistan,2917,0.04,0.00,0.73,2.34,0.96,0.00,2.34
4,Afghanistan,2915,0.04,0.00,0.98,3.05,0.96,0.00,3.05


In [15]:

count_ones = (t_rca_2014['Dexport_dummy'] == 1).sum()
count_zeros = (t_rca_2014['Dexport_dummy'] == 0).sum()
# count_zeros_export_value = (t_rca_2014['export_value'] == 0).sum()

print("cantidad de productos en el 2014:",t_Distance2014['product_code'].nunique())
print("cantidad de países en el 2014:",t_Distance2014['country_name'].nunique()) 
# print("cantidad de países en t_rcaMozam_2014:",t_rcaMozam_2014['country_name'].nunique())
print("cantidad de años en  t_rca_2014:",t_rca_2014['year'].nunique())
print("cantidad de productos en  t_rca_2014:",t_rca_2014['product_code'].nunique())
#print("cantidad de export_value en  t_rca_2014:",t_rca_2014['export_value'].nunique())
# print("cantidad de ceros en export_value en  t_rca_2014:",count_zeros_export_value)




print("Count of 1 (ones) in column: Dexport_dummy:", count_ones)
print("Count of 0 (zeros) in column: Dexport_dummy:", count_zeros)

cantidad de productos en el 2014: 1236
cantidad de países en el 2014: 235
cantidad de años en  t_rca_2014: 1
cantidad de productos en  t_rca_2014: 1236
Count of 1 (ones) in column: Dexport_dummy: 53574
Count of 0 (zeros) in column: Dexport_dummy: 236886


### Cálculo de OGI (2014)

In [16]:
def cal_distancia_normalizada(data,
    ):
  "En la siguiente función se realizará el cálculo de COI para cada país del dataframe"
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de distancia...")

  # t_df['density_r'] = t_df[['Mcp','density']].apply(lambda x: 0 if (x.Mcp==0) else x.density,axis=1 )

  # t_df['distancia']=t_df.apply( lambda x: (1-x.density),axis = 1)
  mean =   t_df['distancia_2014'].mean()
  std =   t_df['distancia_2014'].std()

  t_df['dist_2014_norm']=t_df.apply( lambda x: (x.distancia_2014-mean)/std,axis = 1)

  
  return t_df

def calc_PCI_normalizado(data,
    ):
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades

  mean =   t_df['pci'].mean()
  std =   t_df['pci'].std()
  t_df['pci_norm']=t_df.apply( lambda x: ((x.pci-mean))/std,axis = 1)

  print("Se realiza esuma el right con el COG.")

  return t_df


def calc_densitynormalizado(data,
    ):
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de lado derecho...")

  mean =   t_df['density'].mean()
  std =   t_df['density'].std()
  t_df['density_norm']=t_df.apply( lambda x: ((x.density-mean))/std,axis = 1)

  print("Density normalizado! ")

  return t_df

In [17]:
def cal_ogi_2014(data,
    ):
  "En la siguiente función se realizará el cálculo de COI para cada país del dataframe"
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de lado derecho...")

  # t_df['density_r'] = t_df[['Mcp','density']].apply(lambda x: 0 if (x.Mcp==0) else x.density,axis=1 )

  # t_df['right']=t_df.apply( lambda x: (1-x.dist_norm)*x.pci_norm,axis = 1)
  t_df['right']=t_df.apply( lambda x: (x.density)*x.pci,axis = 1)
  # t_df['right']=t_df.apply( lambda x: (1-x.dist_2014_norm)*x.pci,axis = 1)
  

  densityMean = t_df['density'].mean()
  densityStd = t_df['density'].std()
  t_df['density_norm']=t_df.apply( lambda x: (x.density - densityMean)/densityStd,axis = 1)

  pciMean = t_df['pci'].mean()
  pciStd = t_df['pci'].std()

  t_df['pci_norm']=t_df.apply( lambda x: (x.pci - pciMean)/pciStd,axis = 1)


  t_df['right_Nr']=t_df.apply( lambda x: (x.density_norm)*x.pci,axis = 1)



  mean = t_df['right'].mean()
  std = t_df['right'].std()

  meanCog = t_df['cog'].mean()
  stdCog= t_df['cog'].std()

  print("Se realiza la suma el right con el COG.")
  t_df['cog_norm']=t_df.apply( lambda x: (x.cog-meanCog)/stdCog,axis = 1)

  t_df['right_norm']=t_df.apply( lambda x: (x.right-mean)/std,axis = 1)

  # t_df['ogi_2014'] = t_df.apply(lambda x: x.cog + x.right, axis=1)

  t_df['ogi_2014'] = t_df.apply(lambda x: x.cog, axis=1)

  t_df['ogi_2014_v2'] = t_df.apply(lambda x: x.cog + x.right_Nr, axis=1)

  # meanOgi1 =  t_df['ogi_2014'].mean()
  # meanOgi2 =  t_df['ogi_2014_v2'].mean()

  # stdOgi1 =  t_df['ogi_2014'].std()
  # stdOgi2 =  t_df['ogi_2014_v2'].std()

  # t_df['ogi_2014_norm'] = t_df.apply(lambda x: (x.ogi_2014 -meanOgi1)/stdOgi1, axis=1)
  # t_df['ogi_2014_norm_v2'] = t_df.apply(lambda x: (x.ogi_2014_v2 -meanOgi2)/stdOgi2, axis=1)

  # t_df['ogi2'] = t_df.apply(lambda x: x.cog_norm + x.right_norm, axis=1)
  # meanOgi =  t_df['ogi2'].mean()
  # stdOgi =  t_df['ogi2'].std()
  
  # t_df['ogi2_norm'] = t_df.apply(lambda x: (x.ogi2 -meanOgi)/stdOgi, axis=1)

  t_df.drop(['right_norm','density_norm'],axis=1,inplace=True,errors='ignore')
  print('Cálculo terminado!!')

  return t_df

In [18]:
t_distancia_normalizada = cal_distancia_normalizada(t_Distance2014).copy()

t_ogi2014 = cal_ogi_2014(t_distancia_normalizada).copy()


Se realiza el cálculo de distancia...
Se realiza el cálculo de lado derecho...
Se realiza la suma el right con el COG.
Cálculo terminado!!


In [19]:
t_ogi2014.sample(n=5)
t_ogi2014.describe()

,country_name,product_code,density,rca,cog,pci,distancia_2014,rca_2014,pci_2014,dist_2014_norm,right,pci_norm,right_Nr,cog_norm,ogi_2014,ogi_2014_v2
71651,Czech Republic,5309,0.34,0.63,0.27,2.16,0.66,0.63,2.16,-2.02,0.74,0.70,4.37,-0.32,0.27,4.64
163816,Micronesia,3817,0.01,0.00,0.54,1.19,0.99,0.00,1.19,0.88,0.01,0.14,-1.05,0.28,0.54,-0.51
182309,Niger,8441,0.02,0.00,1.33,3.55,0.98,0.00,3.55,0.78,0.07,1.50,-2.77,2.05,1.33,-1.44
175656,Nepal,8470,0.17,0.00,0.50,1.51,0.83,0.00,1.51,-0.51,0.26,0.33,0.77,0.19,0.50,1.27
146693,Lithuania,2513,0.25,0.13,-0.14,-1.76,0.75,0.13,-1.76,-1.23,-0.44,-1.56,-2.16,-1.24,-0.14,-2.31


,product_code,density,rca,cog,pci,distancia_2014,rca_2014,pci_2014,dist_2014_norm,right,pci_norm,right_Nr,cog_norm,ogi_2014,ogi_2014_v2
count,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00,290460.00
mean,5301.76,0.11,1.61,0.41,0.94,0.89,1.61,0.94,0.00,0.09,0.00,-0.14,-0.00,0.41,0.27
std,2764.80,0.12,63.71,0.45,1.74,0.12,63.71,1.74,1.00,0.29,1.00,1.96,1.00,0.45,1.88
min,101.00,0.00,0.00,-0.79,-4.76,-0.00,0.00,-4.76,-7.73,-2.22,-3.29,-15.95,-2.70,-0.79,-15.95
25%,2843.75,0.03,0.00,0.00,-0.25,0.84,0.00,-0.25,-0.45,-0.01,-0.69,-1.17,-0.92,0.00,-0.59
50%,5401.50,0.07,0.00,0.41,1.04,0.93,0.00,1.04,0.39,0.04,0.06,-0.21,-0.01,0.41,0.09
75%,8101.25,0.16,0.17,0.77,2.26,0.97,0.17,2.26,0.74,0.15,0.76,0.47,0.79,0.77,0.70
max,9999.00,1.00,26347.82,1.55,7.55,1.00,26347.82,7.55,0.96,7.55,3.81,58.40,2.54,1.55,58.40


### Se hace el merge entre la tabla de Dexport Dummy con la tabla ogi/distancia 2014

In [20]:
# t_dexport_rca = pd.merge(t_Distance2014, t_rca_2014[['country_name','product_code','Dexport_dummy']], how="inner", on=['country_name','product_code']).copy()
t_dexport_ogi_rca = pd.merge(t_ogi2014, t_rca_2014[['country_name','product_code','Dexport_dummy']], how="inner", on=['country_name','product_code']).copy()

t_dexport_ogi_rca=t_dexport_ogi_rca[['country_name','product_code','distancia_2014','rca_2014','pci_2014','Dexport_dummy','ogi_2014','ogi_2014_v2']].sort_values(by=['country_name','product_code']).copy()
t_dexport_ogi_rca = t_dexport_ogi_rca.reset_index(drop=True).copy()
t_dexport_ogi_rca.head(n=5)
t_dexport_ogi_rca.sample(n=5)

,country_name,product_code,distancia_2014,rca_2014,pci_2014,Dexport_dummy,ogi_2014,ogi_2014_v2
0,Afghanistan,101,0.94,0.00,0.09,0,0.37,0.33
1,Afghanistan,102,0.92,0.38,-0.81,1,0.08,0.33
2,Afghanistan,103,0.96,0.00,2.25,0,0.77,-0.65
3,Afghanistan,104,0.91,0.00,-2.32,0,-0.11,0.31
4,Afghanistan,105,0.95,0.00,0.75,0,0.56,0.14


,country_name,product_code,distancia_2014,rca_2014,pci_2014,Dexport_dummy,ogi_2014,ogi_2014_v2
231422,Solomon Islands,2825,0.98,0.00,0.21,0,0.36,0.20
279259,Vanuatu,9024,0.99,0.00,2.86,0,1.11,-1.29
175324,Nauru,8484,0.98,0.00,2.93,0,1.07,-1.22
234207,South Africa,5210,0.84,0.11,0.31,1,0.44,0.58
153671,Malaysia,3602,0.81,0.45,-0.67,1,-0.08,-0.56


### Tabla con periodos 2015 - 2018

In [21]:
years = [2014,2015,2016, 2017, 2018]
prepotres = df_complexity[df_complexity['year'].isin(years)].copy()
prepotres = prepotres[['country_name','product_code','export_value','rca','year']].sort_values(by=['year','country_name','product_code']).copy()

prepotres = prepotres.reset_index(drop=True)
prepotres.head()
print(prepotres['country_name'].nunique())
prepotres.info()

,country_name,product_code,export_value,rca,year
0,Afghanistan,101,0,0.00,2014
1,Afghanistan,102,187317,0.38,2014
2,Afghanistan,103,0,0.00,2014
3,Afghanistan,104,0,0.00,2014
4,Afghanistan,105,0,0.00,2014


236
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440607 entries, 0 to 1440606
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   country_name  1440607 non-null  object 
 1   product_code  1440607 non-null  int64  
 2   export_value  1440607 non-null  int64  
 3   rca           1440607 non-null  float64
 4   year          1440607 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 55.0+ MB


### Product Appereance

In [22]:
t_prodApp =  pd.merge(prepotres, t_dexport_ogi_rca[['country_name','product_code','distancia_2014','rca_2014','pci_2014','Dexport_dummy','ogi_2014','ogi_2014_v2']], how="inner", on=['product_code','country_name']).copy()
t_prodApp.head(n=10)
# t_prodApp.sample(n=6)

print(t_prodApp['country_name'].nunique())

,country_name,product_code,export_value,rca,year,distancia_2014,rca_2014,pci_2014,Dexport_dummy,ogi_2014,ogi_2014_v2
0,Afghanistan,101,0,0.00,2014,0.94,0.00,0.09,0,0.37,0.33
1,Afghanistan,101,0,0.00,2015,0.94,0.00,0.09,0,0.37,0.33
2,Afghanistan,101,0,0.00,2016,0.94,0.00,0.09,0,0.37,0.33
3,Afghanistan,101,0,0.00,2017,0.94,0.00,0.09,0,0.37,0.33
4,Afghanistan,101,0,0.00,2018,0.94,0.00,0.09,0,0.37,0.33
5,Afghanistan,102,187317,0.38,2014,0.92,0.38,-0.81,1,0.08,0.33
6,Afghanistan,102,0,0.00,2015,0.92,0.38,-0.81,1,0.08,0.33
7,Afghanistan,102,64743,0.13,2016,0.92,0.38,-0.81,1,0.08,0.33
8,Afghanistan,102,417978,0.84,2017,0.92,0.38,-0.81,1,0.08,0.33
9,Afghanistan,102,130333,0.22,2018,0.92,0.38,-0.81,1,0.08,0.33


235


In [23]:
t_pApp =t_prodApp.copy()
t_pApp['rca_t_1'] = t_pApp.groupby(['country_name','product_code'])['rca'].shift(+1)
t_pApp=t_pApp[t_pApp.year != 2014].copy()
# t_pApp=t_pApp[t_pApp.year != 2015].copy()
# t_pApp=t_pApp[t_pApp.year != 2015].copy()

t_pApp.info()
t_pApp.head(10)

# t_pApp_mozam = t_pApp[t_pApp['country_name']=='Mozambique'].copy()
# t_pApp_mozam.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1147993 entries, 1 to 1438452
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   country_name    1147993 non-null  object 
 1   product_code    1147993 non-null  int64  
 2   export_value    1147993 non-null  int64  
 3   rca             1147993 non-null  float64
 4   year            1147993 non-null  int64  
 5   distancia_2014  1147993 non-null  float64
 6   rca_2014        1147993 non-null  float64
 7   pci_2014        1147993 non-null  float64
 8   Dexport_dummy   1147993 non-null  int64  
 9   ogi_2014        1147993 non-null  float64
 10  ogi_2014_v2     1147993 non-null  float64
 11  rca_t_1         1147993 non-null  float64
dtypes: float64(7), int64(4), object(1)
memory usage: 113.9+ MB


,country_name,product_code,export_value,rca,year,distancia_2014,rca_2014,pci_2014,Dexport_dummy,ogi_2014,ogi_2014_v2,rca_t_1
1,Afghanistan,101,0,0.00,2015,0.94,0.00,0.09,0,0.37,0.33,0.00
2,Afghanistan,101,0,0.00,2016,0.94,0.00,0.09,0,0.37,0.33,0.00
3,Afghanistan,101,0,0.00,2017,0.94,0.00,0.09,0,0.37,0.33,0.00
4,Afghanistan,101,0,0.00,2018,0.94,0.00,0.09,0,0.37,0.33,0.00
6,Afghanistan,102,0,0.00,2015,0.92,0.38,-0.81,1,0.08,0.33,0.38
7,Afghanistan,102,64743,0.13,2016,0.92,0.38,-0.81,1,0.08,0.33,0.00
8,Afghanistan,102,417978,0.84,2017,0.92,0.38,-0.81,1,0.08,0.33,0.13
9,Afghanistan,102,130333,0.22,2018,0.92,0.38,-0.81,1,0.08,0.33,0.84
11,Afghanistan,103,0,0.00,2015,0.96,0.00,2.25,0,0.77,-0.65,0.00
12,Afghanistan,103,0,0.00,2016,0.96,0.00,2.25,0,0.77,-0.65,0.00


In [24]:
def cal_product_appearances(data,
    ):
  t_df = data.copy()
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de product appearances...")

  t_df['product_appearance']=t_df.apply( lambda x: 1 if (x.rca_t_1<1 and x.rca>1) else 0,axis = 1)
  return t_df

In [25]:
t_product_appearance= cal_product_appearances(t_pApp)

Se realiza el cálculo de product appearances...


In [26]:
t_product_appearance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1147993 entries, 1 to 1438452
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   country_name        1147993 non-null  object 
 1   product_code        1147993 non-null  int64  
 2   export_value        1147993 non-null  int64  
 3   rca                 1147993 non-null  float64
 4   year                1147993 non-null  int64  
 5   distancia_2014      1147993 non-null  float64
 6   rca_2014            1147993 non-null  float64
 7   pci_2014            1147993 non-null  float64
 8   Dexport_dummy       1147993 non-null  int64  
 9   ogi_2014            1147993 non-null  float64
 10  ogi_2014_v2         1147993 non-null  float64
 11  rca_t_1             1147993 non-null  float64
 12  product_appearance  1147993 non-null  int64  
dtypes: float64(7), int64(5), object(1)
memory usage: 122.6+ MB


###We run the regression only on products that were absent in 2014. 

In [27]:
# my_lambda = lambda x: x < 1
# t_pro_app_absent = t_product_appearance.copy()
# t_pro_app_absent = t_product_appearance.loc[t_product_appearance['rca_2014'].apply(my_lambda)]
t_pro_app_absent = t_product_appearance.loc[t_product_appearance['rca_2014']<1]

t_pro_app_absent.info()
t_product_appearance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024763 entries, 1 to 1438452
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   country_name        1024763 non-null  object 
 1   product_code        1024763 non-null  int64  
 2   export_value        1024763 non-null  int64  
 3   rca                 1024763 non-null  float64
 4   year                1024763 non-null  int64  
 5   distancia_2014      1024763 non-null  float64
 6   rca_2014            1024763 non-null  float64
 7   pci_2014            1024763 non-null  float64
 8   Dexport_dummy       1024763 non-null  int64  
 9   ogi_2014            1024763 non-null  float64
 10  ogi_2014_v2         1024763 non-null  float64
 11  rca_t_1             1024763 non-null  float64
 12  product_appearance  1024763 non-null  int64  
dtypes: float64(7), int64(5), object(1)
memory usage: 109.5+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1147993 

In [28]:
# t_pro_app_absent.info()
# t_pro_app_absent['product_code'].nunique()
# count_product_appe = (t_pro_app_absent['product_appearance'] == 1).sum()
# t_pro_app_absent.sample(n=10)
# count_product_appe

In [29]:
t_final_col4 = t_pro_app_absent[['country_name','product_code','year','rca_2014','distancia_2014','pci_2014','product_appearance','ogi_2014','ogi_2014_v2']].copy()
t_final_dist_mean = t_final_col4['distancia_2014'].mean()
t_final_dist_std = t_final_col4['distancia_2014'].std()
t_final_pci_mean = t_final_col4['pci_2014'].mean()
t_final_pci_std = t_final_col4['pci_2014'].std()
t_final_ogi_mean = t_final_col4['ogi_2014'].mean()
t_final_ogi_std = t_final_col4['ogi_2014'].std()
t_final_ogi_v2_mean = t_final_col4['ogi_2014_v2'].mean()
t_final_ogi_v2_std = t_final_col4['ogi_2014_v2'].std()

t_final_col4['dist_2014_norm'] = t_final_col4.apply(lambda x: (x.distancia_2014 - t_final_dist_mean)/t_final_dist_std, axis=1)
t_final_col4['pci_2014_norm'] = t_final_col4.apply(lambda x: (x.pci_2014 - t_final_pci_mean)/t_final_pci_std, axis=1)
t_final_col4['ogi_2014_norm_v1'] = t_final_col4.apply(lambda x: (x.ogi_2014 - t_final_ogi_mean)/t_final_ogi_std, axis=1)
t_final_col4['ogi_2014_norm_v2'] = t_final_col4.apply(lambda x: (x.ogi_2014_v2- t_final_ogi_v2_mean)/t_final_ogi_v2_std, axis=1)

t_final_col4.head()

,country_name,product_code,year,rca_2014,distancia_2014,pci_2014,product_appearance,ogi_2014,ogi_2014_v2,dist_2014_norm,pci_2014_norm,ogi_2014_norm_v1,ogi_2014_norm_v2
1,Afghanistan,101,2015,0.00,0.94,0.09,0,0.37,0.33,0.35,-0.52,-0.20,0.11
2,Afghanistan,101,2016,0.00,0.94,0.09,0,0.37,0.33,0.35,-0.52,-0.20,0.11
3,Afghanistan,101,2017,0.00,0.94,0.09,0,0.37,0.33,0.35,-0.52,-0.20,0.11
4,Afghanistan,101,2018,0.00,0.94,0.09,0,0.37,0.33,0.35,-0.52,-0.20,0.11
6,Afghanistan,102,2015,0.38,0.92,-0.81,0,0.08,0.33,0.19,-1.04,-0.86,0.10


In [30]:
t_final_col4.describe()

,product_code,year,rca_2014,distancia_2014,pci_2014,product_appearance,ogi_2014,ogi_2014_v2,dist_2014_norm,pci_2014_norm,ogi_2014_norm_v1,ogi_2014_norm_v2
count,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00
mean,5339.68,2016.50,0.09,0.91,0.99,0.02,0.46,0.17,-0.00,-0.00,0.00,0.00
std,2757.66,1.12,0.19,0.10,1.73,0.13,0.45,1.58,1.00,1.00,1.00,1.00
min,101.00,2015.00,0.00,0.42,-4.76,0.00,-0.79,-13.91,-5.05,-3.33,-2.80,-8.92
25%,2850.00,2015.00,0.00,0.87,-0.19,0.00,0.13,-0.59,-0.41,-0.69,-0.76,-0.48
50%,5407.00,2016.00,0.00,0.94,1.09,0.00,0.48,0.08,0.38,0.06,0.03,-0.06
75%,8109.00,2017.00,0.06,0.98,2.31,0.00,0.81,0.62,0.73,0.76,0.77,0.28
max,9999.00,2018.00,1.00,1.00,7.55,1.00,1.55,30.43,0.97,3.80,2.42,19.18


In [31]:
t_final_col4_tmp = t_final_col4.loc[t_final_col4['rca_2014']<1].copy()
t_final_col4_tmp.describe()
t_final_col4_tmp.info()

,product_code,year,rca_2014,distancia_2014,pci_2014,product_appearance,ogi_2014,ogi_2014_v2,dist_2014_norm,pci_2014_norm,ogi_2014_norm_v1,ogi_2014_norm_v2
count,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00,1024763.00
mean,5339.68,2016.50,0.09,0.91,0.99,0.02,0.46,0.17,-0.00,-0.00,0.00,0.00
std,2757.66,1.12,0.19,0.10,1.73,0.13,0.45,1.58,1.00,1.00,1.00,1.00
min,101.00,2015.00,0.00,0.42,-4.76,0.00,-0.79,-13.91,-5.05,-3.33,-2.80,-8.92
25%,2850.00,2015.00,0.00,0.87,-0.19,0.00,0.13,-0.59,-0.41,-0.69,-0.76,-0.48
50%,5407.00,2016.00,0.00,0.94,1.09,0.00,0.48,0.08,0.38,0.06,0.03,-0.06
75%,8109.00,2017.00,0.06,0.98,2.31,0.00,0.81,0.62,0.73,0.76,0.77,0.28
max,9999.00,2018.00,1.00,1.00,7.55,1.00,1.55,30.43,0.97,3.80,2.42,19.18


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024763 entries, 1 to 1438452
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   country_name        1024763 non-null  object 
 1   product_code        1024763 non-null  int64  
 2   year                1024763 non-null  int64  
 3   rca_2014            1024763 non-null  float64
 4   distancia_2014      1024763 non-null  float64
 5   pci_2014            1024763 non-null  float64
 6   product_appearance  1024763 non-null  int64  
 7   ogi_2014            1024763 non-null  float64
 8   ogi_2014_v2         1024763 non-null  float64
 9   dist_2014_norm      1024763 non-null  float64
 10  pci_2014_norm       1024763 non-null  float64
 11  ogi_2014_norm_v1    1024763 non-null  float64
 12  ogi_2014_norm_v2    1024763 non-null  float64
dtypes: float64(9), int64(3), object(1)
memory usage: 109.5+ MB


## STATS Models in Python

In [32]:
!pip install linearmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.5 MB/s eta 0:00:00


In [33]:
from linearmodels import PanelOLS

data = t_final_col4.set_index(['country_name','year'])

Y_col = ['product_appearance']
X_col = ['dist_2014_norm','pci_2014_norm','ogi_2014_norm_v1']
fe_col = ['country_name','year']

FE = PanelOLS(data[Y_col], data[X_col],
              entity_effects = True,
              time_effects=True
              )
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True )
             # cluster_time=True)
print(result.summary)


# pci: -0.0010
# ogi_2014_norm_v1     0.0017

                          PanelOLS Estimation Summary                           
Dep. Variable:     product_appearance   R-squared:                        0.0050
Estimator:                   PanelOLS   R-squared (Between):             -3.2912
No. Observations:             1024763   R-squared (Within):               0.0050
Date:                Fri, Apr 28 2023   R-squared (Overall):             -0.0694
Time:                        21:39:45   Log-likelihood                 6.422e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1725.8
Entities:                         235   P-value                           0.0000
Avg Obs:                       4360.7   Distribution:               F(3,1024522)
Min Obs:                       1218.0                                           
Max Obs:                       4898.0   F-statistic (robust):             178.70
                            

In [34]:
data.head()

product_code  rca_2014  distancia_2014  pci_2014  product_appearance  ogi_2014  ogi_2014_v2  dist_2014_norm  pci_2014_norm  ogi_2014_norm_v1  ogi_2014_norm_v2
country_name year                                                                                                                                                                
Afghanistan  2015           101      0.00            0.94      0.09                   0      0.37         0.33            0.35          -0.52             -0.20              0.11
             2016           101      0.00            0.94      0.09                   0      0.37         0.33            0.35          -0.52             -0.20              0.11
             2017           101      0.00            0.94      0.09                   0      0.37         0.33            0.35          -0.52             -0.20              0.11
             2018           101      0.00            0.94      0.09                   0      0.37         0.33            0.35          -0.52             -0.20              0.11
             2015           102      0.38            0.92     -0.81                   0      0.08         0.33            0.19          -1.04             -0.86              0.10